# Acquiring Legislation PDFs from Congress.gov

PP project with Shirely and Gifford

In [1]:
import os
import urllib
import pandas as pd

In [2]:
search_results_filename = "search_results_2021-03-19_0151pm.csv"
search_results_filepath = ("data-raw/" + search_results_filename)
search_results_df = pd.read_csv(search_results_filepath, skiprows=3)

search_results_df.head()

,Legislation Number,URL,Congress,Amends Bill,Title,Sponsor,Date of Introduction,Date Offered,Date Submitted,Date Proposed,Number of Cosponsors,Committees,Latest Action Date,Latest Action
0,H.R. 1319,https://www.congress.gov/bill/117th-congress/h...,117th Congress (2021-2022),NaN,American Rescue Plan Act of 2021,"Rep. Yarmuth, John A. [D-KY-3]",02/24/2021,NaN,NaN,NaN,0,House - Budget,03/11/2021,Became Public Law No: 117-2.
1,H.R. 335,https://www.congress.gov/bill/117th-congress/h...,117th Congress (2021-2022),NaN,To provide for an exception to a limitation ag...,"Rep. Smith, Adam [D-WA-9]",01/15/2021,NaN,NaN,NaN,0,House - Armed Services,01/22/2021,Became Public Law No: 117-1.
2,H.R. 8906,https://www.congress.gov/bill/116th-congress/h...,116th Congress (2019-2020),NaN,Lifespan Respite Care Reauthorization Act of 2020,"Rep. Langevin, James R. [D-RI-2]",12/09/2020,NaN,NaN,NaN,1,House - Energy and Commerce,01/05/2021,Became Public Law No: 116-324.
3,H.R. 8900,https://www.congress.gov/bill/116th-congress/h...,116th Congress (2019-2020),NaN,"Further Continuing Appropriations Act, 2021, a...","Rep. Lowey, Nita M. [D-NY-17]",12/08/2020,NaN,NaN,NaN,0,"House - Appropriations, Budget",12/11/2020,Became Public Law No: 116-215.
4,H.R. 8810,https://www.congress.gov/bill/116th-congress/h...,116th Congress (2019-2020),NaN,National Landslide Preparedness Act,"Rep. DelBene, Suzan K. [D-WA-1]",11/24/2020,NaN,NaN,NaN,6,"House - Natural Resources, Science, Space, and...",01/05/2021,Became Public Law No: 116-323.


We are interested in the URL. We will attempt to parse through them and download .PDFs of the bills if available

In [55]:
url = search_results_df.URL #[200] 
url = url.apply(lambda x: x.strip('https://'))
url = url.apply(lambda x: r'https://' + x)
url

0      https://www.congress.gov/bill/117th-congress/h...
1      https://www.congress.gov/bill/117th-congress/h...
2      https://www.congress.gov/bill/116th-congress/h...
3      https://www.congress.gov/bill/116th-congress/h...
4      https://www.congress.gov/bill/116th-congress/h...
                             ...                        
341    https://www.congress.gov/bill/116th-congress/s...
342    https://www.congress.gov/bill/116th-congress/s...
343    https://www.congress.gov/bill/116th-congress/s...
344    https://www.congress.gov/bill/116th-congress/s...
345    https://www.congress.gov/bill/116th-congress/s...
Name: URL, Length: 346, dtype: object

In [56]:
import requests
from bs4 import BeautifulSoup


headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }

In [77]:
# Function to itterate through each of the URLs from the dataframe 
import requests
import time # make sure we sleep to not overwhelm the servers


def acquire_pdf(url, i):
    """
    Acquires PDF of bill from congress.gov
    
    needs url that can be exported from search results. 
    
    I.e. https://www.congress.gov/search?q={%22source%22:%22legislation%22,%22congress%22:[%22117%22,%22116%22],%22bill-status%22:%22law%22}&searchResultViewType=expanded
    """
    
    print(url)
    
    req = requests.get(url, headers)
    soup = BeautifulSoup(req.content, 'html.parser')
    
    time.sleep(4)
    
    links = []
    for a in soup.find_all('td'):
        for b in a.findChildren():
            link = (b.get('href'))
            links.append(link)


    pdf_path_url = [] 
    for l in links:
        if l.endswith(".pdf"):
            pdf_path_url.append(l)

    
    base_path = r'https://www.congress.gov'
    
    try:
        if len(pdf_path_url) < 1:
            print("No PDF found")
        
        else:
            pdf_path = base_path + pdf_path_url[0]

            res = requests.get(pdf_path)

            f = open('data-output/' + 'file_num_' + str(i) + '.pdf', 'wb')
            f.write(res.content)
            f.close()
            print('text saved!')
        
    except:
        print('Unknown error')
    

In [83]:
%%time
url
import timeit

for u, i in zip(url, list(range(len(url)))): 
    acquire_pdf(u, i)

https://www.congress.gov/bill/117th-congress/house-bill/1319
No PDF found
https://www.congress.gov/bill/117th-congress/house-bill/335
No PDF found
https://www.congress.gov/bill/116th-congress/house-bill/8906
No PDF found
https://www.congress.gov/bill/116th-congress/house-bill/8900
text saved!
https://www.congress.gov/bill/116th-congress/house-bill/8810
No PDF found
https://www.congress.gov/bill/116th-congress/house-bill/8611
No PDF found
https://www.congress.gov/bill/116th-congress/house-bill/8472
text saved!
https://www.congress.gov/bill/116th-congress/house-bill/8354
No PDF found
https://www.congress.gov/bill/116th-congress/house-bill/8337
text saved!
https://www.congress.gov/bill/116th-congress/house-bill/8276
text saved!
https://www.congress.gov/bill/116th-congress/house-bill/8247
text saved!
https://www.congress.gov/bill/116th-congress/house-bill/7898
No PDF found
https://www.congress.gov/bill/116th-congress/house-bill/7810
No PDF found
https://www.congress.gov/bill/116th-congress

No PDF found
https://www.congress.gov/bill/116th-congress/house-bill/3245
text saved!
https://www.congress.gov/bill/116th-congress/house-bill/3207
text saved!
https://www.congress.gov/bill/116th-congress/house-bill/3196
text saved!
https://www.congress.gov/bill/116th-congress/house-bill/3153
No PDF found
https://www.congress.gov/bill/116th-congress/house-bill/3151
text saved!
https://www.congress.gov/bill/116th-congress/house-bill/3144
text saved!
https://www.congress.gov/bill/116th-congress/house-bill/3055
text saved!
https://www.congress.gov/bill/116th-congress/house-bill/3005
text saved!
https://www.congress.gov/bill/116th-congress/house-bill/2969
text saved!
https://www.congress.gov/bill/116th-congress/house-bill/2940
text saved!
https://www.congress.gov/bill/116th-congress/house-bill/2938
text saved!
https://www.congress.gov/bill/116th-congress/house-bill/2744
No PDF found
https://www.congress.gov/bill/116th-congress/house-bill/2695
text saved!
https://www.congress.gov/bill/116th-

ConnectionError: HTTPSConnectionPool(host='www.congress.govhttps', port=443): Max retries exceeded with url: //www.govinfo.gov/content/pkg/CREC-2020-12-21/pdf/CREC-2020-12-21-house-bk3.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001945C7E84C8>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [104]:
%%time 

urls_sub = url[216:]

for u, i in zip(urls_sub, list(range(len(urls_sub), len(url)))): 
    acquire_pdf(u, i)

https://www.congress.gov/bill/116th-congress/house-joint-resolution/110
text saved!
https://www.congress.gov/bill/116th-congress/house-joint-resolution/107
text saved!
https://www.congress.gov/bill/116th-congress/house-joint-resolution/88
text saved!
https://www.congress.gov/bill/116th-congress/house-joint-resolution/87
text saved!
https://www.congress.gov/bill/116th-congress/house-joint-resolution/80
text saved!
https://www.congress.gov/bill/116th-congress/house-joint-resolution/60
text saved!
https://www.congress.gov/bill/116th-congress/house-joint-resolution/31
text saved!
https://www.congress.gov/bill/116th-congress/house-joint-resolution/28
text saved!
https://www.congress.gov/bill/116th-congress/senate-bill/5076
No PDF found
https://www.congress.gov/bill/116th-congress/senate-bill/5036
text saved!
https://www.congress.gov/bill/116th-congress/senate-bill/4996
No PDF found
https://www.congress.gov/bill/116th-congress/senate-bill/4902
text saved!
https://www.congress.gov/bill/116th-

text saved!
https://www.congress.gov/bill/116th-congress/senate-bill/256
text saved!
https://www.congress.gov/bill/116th-congress/senate-bill/252
text saved!
https://www.congress.gov/bill/116th-congress/senate-bill/239
text saved!
https://www.congress.gov/bill/116th-congress/senate-bill/227
text saved!
https://www.congress.gov/bill/116th-congress/senate-bill/216
text saved!
https://www.congress.gov/bill/116th-congress/senate-bill/212
No PDF found
https://www.congress.gov/bill/116th-congress/senate-bill/209
text saved!
https://www.congress.gov/bill/116th-congress/senate-bill/199
text saved!
https://www.congress.gov/bill/116th-congress/senate-bill/163
text saved!
https://www.congress.gov/bill/116th-congress/senate-bill/153
text saved!
https://www.congress.gov/bill/116th-congress/senate-bill/151
text saved!
https://www.congress.gov/bill/116th-congress/senate-bill/134
text saved!
https://www.congress.gov/bill/116th-congress/senate-bill/50
text saved!
https://www.congress.gov/bill/116th-con

In [100]:
urls_sub = url[215:]

print(range(215, len(url)))

range(215, 346)


In [48]:
url = r'https://www.congress.gov/bill/116th-congress/house-bill/8900' # url[105]
# url = url.strip('https://')
# url = r'https://' + str(url)
url


req = requests.get(url, headers)
soup = BeautifulSoup(req.content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html class="no-js" lang="en">
 <head>
  <title>
   H.R.8900 - 116th Congress (2019-2020): Further Continuing Appropriations Act, 2021, and Other Extensions Act | Congress.gov | Library of Congress
  </title>
  <meta charset="utf-8"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="https://www.congress.gov/bill/116th-congress/house-bill/8900" name="canonical"/>
  <meta content="2019/2020" name="dc.coverage"/>
  <meta content="2020-12-08" name="dc.coverage"/>
  <meta content="12/11/2020" name="dc.coverage"/>
  <meta content="Lowey, Nita M." name="dc.creator"/>
  <meta content="12/11/2020" name="dc.date"/>
  <meta content="https://www.congress.gov/bill/116th-congress/house-bill/8900" name="dc.identifier"/>
  <meta content="/bill/116th-congress/house-bill/8900" name="dc.identifier"/>
  <meta content="eng" name="dc.language"/>
  <meta content="Text is government work" name="dc.rights"/>
  <meta content="House Bill" name="dc.subject"/>

In [50]:
links = []

for a in soup.find_all('td'):
    for b in a.findChildren():
        link = (b.get('href'))
        links.append(link)

print(links)

pdf_path_url = [] 
for l in links:
    if l.endswith(".pdf"):
        pdf_path_url.append(l)

pdf_path_url

['/member/nita-lowey/L000480', '/bill/116th-congress/house-bill/8900/text?overview=closed', '/116/plaws/publ215/PLAW-116publ215.pdf', '/bill/116th-congress/house-bill/8900/all-actions?overview=closed#tabs', '/bill/116th-congress/house-bill/8900/all-actions?overview=closed&q=%7B%22roll-call-vote%22%3A%22all%22%7D']


['/116/plaws/publ215/PLAW-116publ215.pdf']

In [51]:
import requests

base_path = r'https://www.congress.gov'
pdf_path = base_path + pdf_path_url[0]

res = requests.get(pdf_path)


In [52]:
print(res)
f = open('data-raw/test3.pdf', 'wb')
f.write(res.content)
f.close()

<Response [200]>


In [76]:
print(list(range(5)))

[0, 1, 2, 3, 4]
